In [24]:
import pandas as pd
import random
import uuid
from datetime import datetime

def generate_unique_employee_id():
    return "E" + str(uuid.uuid4()).split('-')[0]  # e.g. E4a59bf4e

# Add a new employee (error if EmployeeID provided)
def add_employee(employees_df, new_employee):
    if 'EmployeeID' in new_employee and new_employee['EmployeeID']:
        raise ValueError("New employee must not have an EmployeeID; it will be autogenerated.")
    new_employee['EmployeeID'] = generate_unique_employee_id()
    employees_df = pd.concat([employees_df, pd.DataFrame([new_employee])], ignore_index=True)
    print(f"Added new employee with ID {new_employee['EmployeeID']}")
    employees_df.to_csv('/content/Employee_with_ID_with_random_skills (1).csv', index=False)
    return employees_df

# Update an existing employee by EmployeeID
def update_employee(employees_df, employee_id, updated_fields):
    if employee_id not in employees_df['EmployeeID'].values:
        raise ValueError(f"EmployeeID {employee_id} not found. Cannot update non-existent employee.")
    for key, value in updated_fields.items():
        employees_df.loc[employees_df['EmployeeID'] == employee_id, key] = value
    print(f"Updated employee {employee_id} with fields {updated_fields}")
    employees_df.to_csv('/content/Employee_with_ID_with_random_skills (1).csv', index=False)
    return employees_df

# Delete an employee by EmployeeID
def delete_employee(employees_df, employee_id):
    if employee_id not in employees_df['EmployeeID'].values:
        raise ValueError(f"EmployeeID {employee_id} not found. Cannot delete non-existent employee.")
    employees_df = employees_df.drop(employees_df[employees_df['EmployeeID'] == employee_id].index)
    print(f"Deleted employee with ID {employee_id}")
    employees_df.to_csv('/content/Employee_with_ID_with_random_skills (1).csv', index=False)
    return employees_df

# Interactive delete employee
def delete_employee_interactive(employees_df):
    employee_id = input("Enter EmployeeID to delete or type 'no' to skip deletion: ").strip()
    if employee_id.lower() == 'no':
        print("Deletion skipped.")
        return employees_df
    if employee_id not in employees_df['EmployeeID'].values:
        print(f"EmployeeID {employee_id} not found. No deletion performed.")
        return employees_df
    employees_df = employees_df.drop(employees_df[employees_df['EmployeeID'] == employee_id].index)
    print(f"Deleted employee with ID {employee_id}")
    employees_df.to_csv('/content/Employee_with_ID_with_random_skills (1).csv', index=False)
    return employees_df

# Interactive add project
def add_project_interactive(projects_df):
    user_input = input("Do you want to add a new project? (yes/no): ").strip().lower()
    if user_input != 'yes':
        print("Skipping adding new project.")
        return projects_df

    print("Enter new project details:")
    project_name = input("Project Name: ").strip()
    deadline = input("Deadline (YYYY-MM-DD): ").strip()
    sub_task = input("Sub Task: ").strip()
    skills_required = input("Skills Required (comma-separated): ").strip()

    new_project = {
        'Project Name': project_name,
        'Deadline': deadline,
        'Sub Task': sub_task,
        'Skills Required': skills_required
    }

    projects_df = pd.concat([projects_df, pd.DataFrame([new_project])], ignore_index=True)
    projects_df.to_csv('/content/project_tasks_500.csv', index=False)
    print(f"Added new project '{project_name}' with sub task '{sub_task}' and saved to CSV.")

    return projects_df

# Load employees and projects CSV files
employees = pd.read_csv('/content/Employee_with_ID_with_random_skills (1).csv')
projects = pd.read_csv('/content/project_tasks_500.csv')

# Interactive section to add a new employee
user_input = input("Do you want to add a new employee? (yes/no): ").strip().lower()
if user_input == 'yes':
    new_employee = {}
    new_employee['Education'] = input("Enter Education: ")
    new_employee['JoiningYear'] = int(input("Enter Joining Year: "))
    new_employee['City'] = input("Enter City: ")
    new_employee['PaymentTier'] = int(input("Enter Payment Tier: "))
    new_employee['Age'] = int(input("Enter Age: "))
    new_employee['Gender'] = input("Enter Gender: ")
    new_employee['EverBenched'] = input("Ever Benched (Yes/No): ")
    new_employee['ExperienceInCurrentDomain'] = int(input("Enter Experience in Current Domain (years): "))
    new_employee['LeaveOrNot'] = int(input("Leave or Not (0/1): "))
    new_employee['skills'] = input("Enter skills (comma separated): ")
    employees = add_employee(employees, new_employee)

# Sort projects by earliest deadline for assignment
projects_sorted = projects.sort_values(by='Deadline').reset_index(drop=True)

# Assign tasks avoiding repeat project assignments for employees
assignments = []
employee_project_map = {}  # EmployeeID -> set of assigned projects

for i, task in projects_sorted.iterrows():
    available_employees = employees['EmployeeID'].tolist()
    available_employees = [
        e for e in available_employees
        if task['Project Name'] not in employee_project_map.get(e, set())
    ]
    if not available_employees:
        assigned_employee = None
    else:
        assigned_employee = random.choice(available_employees)
        employee_project_map.setdefault(assigned_employee, set()).add(task['Project Name'])

    assignments.append({
        'Project Name': task['Project Name'],
        'Deadline': task['Deadline'],
        'Sub Task': task['Sub Task'],
        'Skills Required': task['Skills Required'],
        'Assigned EmployeeID': assigned_employee
    })

# Save assignments to CSV
df_assignments = pd.DataFrame(assignments)
df_assignments.to_csv('project_task_assignments.csv', index=False)
print("✅ Assignments saved to project_task_assignments.csv")

# Example update employee usage
employee_id_to_update = input("Enter EmployeeID to update or 'no' to skip: ").strip()
if employee_id_to_update.lower() == 'no':
    print("Update skipped.")
elif employee_id_to_update:
    updated_fields = {}
    updated_fields['City'] = input("Enter updated City: ")
    updated_fields['PaymentTier'] = int(input("Enter updated Payment Tier: "))
    updated_fields['skills'] = input("Enter updated skills (comma separated): ")
    employees = update_employee(employees, employee_id_to_update, updated_fields)
else:
    print("Employee ID to update not provided.")

# Delete an employee interactively
employees = delete_employee_interactive(employees)

# Interactive option to add new project
projects = add_project_interactive(projects)

# Attempt to load completed projects CSV file
try:
    completed_projects = pd.read_csv('/content/completed_projects.csv')
except FileNotFoundError:
    completed_projects = pd.DataFrame(columns=projects.columns)

# Convert 'Deadline' to datetime
projects['Deadline'] = pd.to_datetime(projects['Deadline'], format='mixed')

# Get today's date as Pandas Timestamp for comparison
today = pd.Timestamp(datetime.now().date())

# Separate completed vs active projects by comparing Deadline with today
completed_mask = projects['Deadline'] < today
completed_now = projects[completed_mask]
projects = projects[~completed_mask]

# Append newly completed projects to completed_projects DataFrame
completed_projects = pd.concat([completed_projects, completed_now], ignore_index=True)

# Save updated DataFrames back to CSV
projects.to_csv('/content/project_tasks_500.csv', index=False)  # active/incomplete projects
completed_projects.to_csv('/content/completed_projects.csv', index=False)  # completed projects

print(f"Moved {len(completed_now)} completed projects to completed_projects.csv")

Do you want to add a new employee? (yes/no): no
✅ Assignments saved to project_task_assignments.csv
Enter EmployeeID to update or 'no' to skip: no
Update skipped.
Enter EmployeeID to delete or type 'no' to skip deletion: no
Deletion skipped.
Do you want to add a new project? (yes/no): yes
Enter new project details:
Project Name: computer
Deadline (YYYY-MM-DD): 2025-10-3
Sub Task: preprocessing
Skills Required (comma-separated): java
Added new project 'computer' with sub task 'preprocessing' and saved to CSV.
Moved 0 completed projects to completed_projects.csv


In [13]:
assigned_tasks=pd.read_csv("/content/project_task_assignments.csv")
are_all_unique = assigned_tasks['Assigned EmployeeID'].dropna().is_unique

print(are_all_unique)

False
